In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import collections
import copy as cp
%matplotlib inline

In [ ]:
economic_datasets = collections.OrderedDict([
 
    ('real_gdp_growth_rate', {'filename': 'data/A191RL1Q225SBEA_real_gross_domestic_product.csv',
            'column_rename': {'A191RL1Q225SBEA': 'real_gdp_growth_rate'}, 'ylabel': 'Real annual GDP growth rate'}),
    ('sticky_inflation_rate', {'filename': 'data/CORESTICKM158SFRBATL_sticky_consumer_price_index_annual_rate_of_change_less_food_and_energy.csv',
            'column_rename': {'CORESTICKM158SFRBATL': 'sticky_inflation_rate'}, 'ylabel': 'Annual inflation rate (ignoring food and energy)'}),
    ('federal_deficit_percent_gdp', {'filename': 'data/FYFSGDA188S_federal_surplus_or_deficit_as_percent_of_gdp.csv',
            'column_rename': {'FYFSGDA188S': 'federal_deficit_percent_gdp'}, 'ylabel': 'Annual federal deficit (% GDP)'}),
    ('nonfarm_job_growth_monthly_thousands', {'filename': 'data/PAYEMS_total_nonfarm_jobs.csv', 'column_rename':\
            {'PAYEMS': 'nonfarm_jobs_thousands'}, 'ylabel': 'Monthly nonfarm job growth rate (thousands)',
            'apply_delta_to_column': 'nonfarm_jobs_thousands'}),
#     ('u3_unemployment', {'filename': 'data/UNRATE_civilian_unemployment_rate_U3.csv'}),
#     ('u6_unemployment', {'filename': 'data/U6UNEM6AR_u6_unemployment_rate.csv'}),

])



In [ ]:
def make_time_series_plot(input_df, y_column, xlabel, ylabel, xlim, ylim):
    plt.figure(figsize=(16,8))
    plt.grid(True)
    plt.plot(input_df['DATE'], input_df[y_column])
    plt.xlabel(xlabel, fontsize=22)
    plt.ylabel(ylabel, fontsize=22)
    if xlim is not None:
        plt.gca().set_xlim(xlim)
    if ylim is not None:
        plt.gca().set_ylim(ylim)
    plt.plot()

In [ ]:
combined_df = None
for metric_key, metric_instructions in economic_datasets.iteritems():
    print metric_key, ', instructions: ', metric_instructions
    this_df = pd.read_csv(metric_instructions['filename'])
    this_df['DATE'] = pd.to_datetime(this_df['DATE'])
    this_df = this_df.rename(columns=metric_instructions['column_rename'])
#     if metric_instructions.get('apply_delta', False):
#         this_df[metric_key] = this_df[]
    if metric_instructions.get('apply_delta_to_column', None) is not None:
        print 'Do Xform of ', this_df.head(10)
        this_df[metric_key] = this_df[metric_instructions['apply_delta_to_column']].diff()
        #this_df[metric_key] = this_df.apply(metric_instructions['transform_function'], axis=1)
    print 'this_df: ', this_df.head(10)
    
    if combined_df is None:
        combined_df = cp.deepcopy(this_df)
    else:
        combined_df = pd.merge(combined_df, this_df, on='DATE', how='outer')
combined_df = combined_df.sort_values('DATE')
print 'combined_df: ', combined_df

In [ ]:
def make_time_series_plot(input_df, y_column, ylabel, xlim, ylim):
    filtered_input_df = input_df[pd.notnull(input_df[y_column])]
    plt.figure(figsize=(16,8))
    plt.grid(True)
    print 'Plot ', filtered_input_df[['DATE', y_column]]
    plt.plot(filtered_input_df['DATE'].values, filtered_input_df[y_column].values)
    plt.xlabel('Date', fontsize=22)
    plt.ylabel(ylabel, fontsize=22)
    if xlim is not None:
        plt.gca().set_xlim(xlim)
    if ylim is not None:
        plt.gca().set_ylim(ylim)
    plt.plot()

In [ ]:
for metric_key, metric_instructions in economic_datasets.iteritems():
    print 'metric_instructions: ', metric_instructions
    make_time_series_plot(combined_df, y_column=metric_key, ylabel=metric_instructions['ylabel'], xlim=None, ylim=None)